In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from shutil import copyfile

In [ ]:
class Axes:
    
    def __init__(self, filename):
        # initialise the axes by reading data from the metadata (.meta.txt) file
        self.process = True
        with open(filename) as f:
            lines = [line.rstrip('\n') for line in open(filename)]
        if lines[0] != '#inner loop':
            print("problem initialising Axes: inner loop "+filename)
            self.process = False
        else:
            self.no_inner_loop = int(lines[1])
            self.min_inner_loop = lines[2]
            self.max_inner_loop = lines[3]
            self.label_inner_loop = lines[4]
            if lines[5] != '#outer loop':
                print("problem initialising Axes: outer loop "+filename)
                self.process = False
            else:
                self.no_outer_loop = int(lines[6])
                self.min_outer_loop = lines[7]
                self.max_outer_loop = lines[8]
                self.label_outer_loop = lines[9]
                if lines[10] != '#outer most loop':
                    self.no_outer_most_loop = 0
                    self.min_outer_most_loop = 0
                    self.max_outer_most_loop = 0
                    self.label_outer_most_loop = ""
                else:
                    self.no_outer_most_loop = int(lines[11])
                    self.min_outer_most_loop = lines[12]
                    self.max_outer_most_loop = lines[13]
                    self.label_outer_most_loop = lines[14]

In [ ]:
class Data:
    
    def __init__(self, filename):
        # initialise the experimental data by reading the header from the data (.dat) file
        self.process = True # should this experiment be processed?
        self.axes = []      # axes labels
        self.columns = {}   # column labels and index numbers
        with open(filename) as f:
            lines = [line.rstrip('\n') for line in open(filename)]
            line = 3
            col_no = 1
            while lines[line].find("Column") != -1:
                if lines[line+3].find("type: coordinate") > 0:
                    axis_text = lines[line+1][lines[line+1].find(": ")+2:]
                    self.axes.append(axis_text)
                    line += 4
                elif lines[line+2].find("type: value") > 0:
                    column_text = lines[line+1][lines[line+1].find(": ")+2:]
                    self.columns[column_text]=col_no
                    line += 3
                col_no += 1
        if len(self.axes) < 2: self.process = False

In [ ]:
class File_set:
    
    def __init__(self, sourceDir, filename):
        # locate the dat sets from the .dat file, do sanity check and copy the complete data sets to their target destination
        self.input_files_present = False
        if sourceDir.startswith(sourceTree) and filename.endswith(".dat"):
            no_dat = 0
            no_py = 0
            for file in os.listdir(sourceDir): # if there is more than 1 .dat file in a directory, create an extra directory
                if file.endswith(".dat"):
                    no_dat +=1
                elif file.endswith(".py"):
                    self.pyfile = file
                    no_py +=1
            if no_dat > 1:
                self.extra_dir = filename[:-4]
            else:
                self.extra_dir = ""
            
            self.datfile = filename
            file_base = self.datfile[:-4]
            path_base = os.path.join(sourceDir,filename)[:-4]
            if os.path.exists(path_base+".meta.txt"):
                self.metafile = file_base+".meta.txt"
                if os.path.exists(path_base+".set"):
                    self.setfile = file_base+".set"
                    self.input_files_present = True

                    if os.path.exists(path_base+".py"):
                        self.pyfile = file_base+".py"
                    elif no_py != 1:
                        self.pyfile = ""
        else:
            print("problem initialising the input files: invalid calling parameters")        
        
    def copy_file_set(self, sourceTree, destTree):
        success = False
        if sourceDir.startswith(sourceTree):
            success = True
            destDir = destTree+sourceDir[len(sourceTree):]
            if not os.path.exists(destDir): os.makedirs(destDir)
            if self.extra_dir != "":
                destDir = os.path.join(destDir,self.extra_dir)
                if not os.path.exists(destDir): os.makedirs(destDir)
            self.destDir = destDir
            if not os.path.isfile(os.path.join(destDir,self.datfile)):
                try:
                    copyfile(os.path.join(sourceDir,self.datfile), os.path.join(destDir,self.datfile))
                except:
                    success = False
            if not os.path.isfile(os.path.join(destDir,self.metafile)):
                try:
                    copyfile(os.path.join(sourceDir,self.metafile), os.path.join(destDir,self.metafile))
                except:
                    success = False
            if not os.path.isfile(os.path.join(destDir,self.setfile)):
                try:
                    copyfile(os.path.join(sourceDir,self.setfile), os.path.join(destDir,self.setfile))
                except:
                    success = False
            if self.pyfile != "" and not os.path.isfile(os.path.join(destDir,self.pyfile)):
                try:
                    copyfile(os.path.join(sourceDir,self.pyfile), os.path.join(destDir,self.pyfile))
                except:
                    success = False
        return success   

In [ ]:
def generate_plots(base, sourceDir, destDir, axes, data):

    no_plots = 0
    datfile = os.path.join(sourceDir,base)+".dat"
    toBase = os.path.join(destDir,base)+".dat"
    plot3D = len(data.axes) == 3
    if plot3D:
        xdata = np.genfromtxt(datfile, usecols=(0))
        ydata = np.genfromtxt(datfile, usecols=(1))
        zdata = np.genfromtxt(datfile, usecols=(2))
    for column_text, col in data.columns.items():
        no_plots +=1
        if not os.path.exists(toBase+'_plot_'+str(col)+'.jpg'): # don't overwrite
            data = np.genfromtxt(datfile, usecols=(col-1))
            fig = plt.figure()
            if plot3D:
                ax = fig.add_subplot(111, projection='3d')
                extremes = sorted(data)
                data_points = len(extremes)
                no_to_include = int(data_points/100.)
                large_min = extremes[no_to_include]
                small_max = extremes[data_points-1-no_to_include]

                xplot=[]
                yplot=[]
                zplot=[]
                cplot=[]
                for index, value in enumerate(data):
                    if value <= large_min or value >= small_max:
                        xplot.append(xdata[index])
                        yplot.append(ydata[index])
                        zplot.append(zdata[index])
                        cplot.append(data[index])

                im = ax.scatter3D(xplot, yplot, zplot, c=cplot, cmap=None)

                #ax.set_xticks([axes.min_inner_loop, axes.max_inner_loop])
                #ax.set_yticks([axes.min_outer_loop, axes.max_outer_loop])
                
                ax.set_zlabel(axes.label_outer_most_loop)
                #ax.set_zticks([axes.min_outer_most_loop, axes.max_outer_most_loop])
                #ax.set_zticklabels([axes.min_outer_loop, axes.max_outer_most_loop])

            else:
                heatmap = np.resize(data,(axes.no_outer_loop,axes.no_inner_loop))
    
                ax = fig.add_subplot(111)
                im = ax.imshow(heatmap, interpolation='nearest', cmap=None)
                ax.set_xticks([0, axes.no_inner_loop])
                ax.set_xticklabels([axes.min_inner_loop, axes.max_inner_loop])
                ax.set_yticks([0, axes.no_outer_loop])
                ax.set_yticklabels([axes.min_outer_loop, axes.max_outer_loop])

            ax.set_xlabel(axes.label_inner_loop)
            ax.set_ylabel(axes.label_outer_loop)

            ax.set_title(column_text)

            # Add a colour bar along the bottom and label it
            cbar = fig.colorbar(ax=ax, mappable=im, orientation='horizontal')
            cbar.set_label(base) #looks better this way

            #plt.show()
            plt.savefig(toBase+'_plot_'+str(col)+'.jpg')
            plt.close()
    return no_plots

In [ ]:
def add_metadata(imetafile, setfile, destDir, no_plots):
    
    with open(setfile) as f:
        lines = [line.rstrip('\n') for line in open(setfile)]
    instrument = ""
    for line in lines:
        try:
            tag, value = line.split(": ")
        except:
            continue
        tag.strip()
        tag = tag.replace("\t", "")
        if tag == "Instrument":
            instrument = value+"_"
        else:
            tag.strip()
            #SEB if value.strip() != "None":
            imetafile.write("imeta add -C "+str(destDir)+" "+instrument+tag+" '"+value+"' '' \n") #SEB added '' unit
    #SEB imetafile.write("imeta add -C "+str(destDir)+" number_of_plots '"+str(no_plots)+"' '' \n") #SEB added '' unit

In [ ]:
# main calling loop
sourceTree = 'D:' #Source of files and metadata
destTree = 'Y:/garys' #iRODS destination - mapped as disc drive with davrods
if not os.path.exists(destTree): os.makedirs(destTree)
metadata_file = open(os.path.join(destTree,"metadata.txt"),"w")

In [ ]:
todo_list=[]
for sourceDir, subdirList, fileList in os.walk(sourceTree):

    for fname in fileList:
        #print('\t%s' % fname)
        no_plots = 0
        if fname.endswith(".dat"):
            fs = File_set(sourceDir, fname)
            if fs.input_files_present:
                make_jpgs = fs.copy_file_set(sourceTree, destTree)
                try:
                    a = Axes(os.path.join(sourceDir,fs.metafile))
                    if a.process:
                        d = Data(os.path.join(sourceDir,fs.datfile))
                except:
                    make_jpgs = False
                if make_jpgs and a.process and d.process:
                    consistent = True
                    if d.axes[0] != a.label_inner_loop or d.axes[1] != a.label_outer_loop:
                        print("problem: loop information inconsistent "+os.path.join(sourceDir,fs.metafile))
                        consistent = False
                    if len(d.axes) == 3:
                        if d.axes[2] != a.label_outer_most_loop:
                            print("problem: 3rd axis information inconsistent "+os.path.join(sourceDir,fs.metafile))
                            consistent = False
                    if consistent == True:
                        todo_list.append(fname)
                        no_plots = generate_plots(fname[:-4], sourceDir, fs.destDir, a, d)
                add_metadata(metadata_file, os.path.join(sourceDir,fs.setfile), fs.destDir, no_plots)
                # add_metadata only possible if copy_file_set has been run first
            else:
                print("problem: missing input files "+os.path.join(sourceDir,fname))
                continue

In [ ]:
print(len(todo_list))

In [ ]:
metadata_file.close()